In [ ]:
import sqlalchemy as sql
from sqlalchemy_utils import database_exists
import psycopg2
import networkx as nx
from sqlalchemy import Integer, BigInteger, Float, Column
from geoalchemy2 import Geometry, WKTElement
from geoalchemy2 import select
import pandas as pd
import geopandas as gpd

In [ ]:
hostname = "wheelway2.cgfv5tiyps6x.us-east-1.rds.amazonaws.com"
username = "postgres"
with open('/home/adam/rdskey') as keyfile:
    rds_key = keyfile.readline().strip()
    
con = psycopg2.connect(database = "wheelway", 
                       user=username, 
                       host=hostname, 
                       password=rds_key, 
                       port=5432
                      )

engine = sql.create_engine('postgresql+psycopg2://{user}:{pwd}@{host}:{port}/wheelway'.format(user=username,
                                                                                     pwd=rds_key,
                                                                                     host=hostname,
                                                                                     port=5432
                                                                                     ))

In [ ]:
cur = con.cursor()

In [57]:
def fixed_route(rows):
    #dedupe and keep order
    rows = list(dict.fromkeys(rows))
    first_row = rows.pop(0)
    relinked = [(first_row[2],first_row[3], first_row[1])]
    for row in rows:
        row = (row[1], row[2], row[3])
        last_row = relinked[-1]
        if row == last_row:
            continue
        elif last_row[2] == row[1]:
            relinked.append(row)
        else:
            relinked.append((row[2], row[1], row[0]))
    return relinked

# why do we need to split this ourselves??
def query_route(ori_int, des_int):
    cur.execute("SELECT node, b.angle_class, ST_AsText(ST_StartPoint(b.geom)), ST_AsText(ST_EndPoint(b.geom)), ST_AsText(b.geom) FROM pgr_dijkstra('SELECT osmid AS id, source, target, cost FROM brighton_edges', %s, %s, directed:=true) a LEFT JOIN brighton_edges b ON (a.edge = b.osmid)", (ori_int, des_int))
    raw_route = cur.fetchall()
    return fixed_route(raw_route)

In [62]:
test = query_route(163469, 21552)

IndexError: pop from empty list

In [59]:
test

[('POINT(-71.1604252361324 42.3473344952673)',
  'POINT(-71.1604233138344 42.3473138188153)',
  0),
 ('POINT(-71.1604233138344 42.3473138188153)',
  'POINT(-71.1604147287036 42.3473112789833)',
  4),
 ('POINT(-71.1604147287036 42.3473112789833)',
  'POINT(-71.1603926438479 42.3473044908492)',
  4),
 ('POINT(-71.1603926438479 42.3473044908492)',
  'POINT(-71.1603693457622 42.3472977255787)',
  4),
 ('POINT(-71.1603693457622 42.3472977255787)',
  'POINT(-71.1603472609159 42.3472909374356)',
  4),
 ('POINT(-71.1603472609159 42.3472909374356)',
  'POINT(-71.1603251760742 42.3472841492882)',
  3),
 ('POINT(-71.1603251760742 42.3472841492882)',
  'POINT(-71.1603018780028 42.3472773840036)',
  3),
 ('POINT(-71.1603018780028 42.3472773840036)',
  'POINT(-71.1602797931704 42.3472705958472)',
  2),
 ('POINT(-71.1602797931704 42.3472705958472)',
  'POINT(-71.1602577083427 42.3472638076864)',
  2),
 ('POINT(-71.1602577083427 42.3472638076864)',
  'POINT(-71.1602344102855 42.3472570423877)',
  2),


In [41]:
cur.execute("""
SELECT id, the_geom <-> ST_SetSRID(ST_MakePoint(42.35, -71.16), 4326) AS dist 
FROM brighton_edges_noded_vertices_pgr
ORDER BY dist LIMIT 1;""")

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [38]:
pt_string = cur.fetchall()

In [42]:
con.rollback()

In [31]:
pt_string = pt_string[0][0]

In [36]:
[float(z) for z in pt_string[6:-1].split()]

[-71.1378814839898, 42.343913648335]

In [39]:
pt_string

[(27004, 160.507438012115)]